In [1]:
# !pip install selenium requests webdriver_manager bs4

In [2]:
!curl https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -o google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install --fix-broken
!google-chrome --version

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  102M  100  102M    0     0  1739k      0  0:01:00  0:01:00 --:--:-- 1805k
dpkg: error: requested operation requires superuser privilege
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Google Chrome 121.0.6167.184 


In [3]:
!wget https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!ls

--2024-04-03 22:33:23--  https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 172.217.170.187, 172.217.170.219, 2a00:1450:401a:800::201b, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|172.217.170.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7315061 (7.0M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   6.98M  1.67MB/s    in 4.0s    

2024-04-03 22:33:28 (1.72 MB/s) - ‘chromedriver_linux64.zip’ saved [7315061/7315061]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
  inflating: LICENSE.chromedriver    
24crawler.ipynb				housepricedata.csv
chromedriver				LICENSE.chromedriver
chromedriver_linux64.zip		README.md
google-chrome-stable_current_amd64.deb	requirements.txt
house_data_prediction.ipynb		web_crawler.ipynb

! chmod +x chromedriver
!sudo mv chromedriver /usr/local/bin/
# !rm -rf /usr/local/bin/chromedriver
!ls /usr/local/bin


In [5]:
import datetime
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

def parse():
    base_url = "https://www.pararius.com/apartments/amsterdam"

    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode to avoid opening a browser window
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())

    try:
        print("Getting driver...")
        # Launch the Chrome browser with the configured options
        driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)
        

        print("Done!")
    except Exception as err:
        print("Encountered an exception!")
        raise err

    time.sleep(5)
    # Navigate to the URL
    driver.get(base_url)

    # Use explicit wait to wait for recipe elements to load
    wait = WebDriverWait(driver, 7)
    time.sleep(10)

    # Get the page source (which will contain the dynamically generated content)
    page_source = driver.page_source

    # Extract recipes from the page source
    houses = extract(page_source)
    # print(f"Houses: {houses}")

    driver.quit() # close driver

def extract(page_source) -> dict:
    # Use BeautifulSoup to parse the page source and extract the data you need
    soup = BeautifulSoup(page_source, 'html.parser')
    print(f"This is kado: \n{soup}")

if __name__ == "__main__":
    parse()


Getting driver...
Done!
This is kado: 
<html class="has-js" lang="en"><head><meta content="Az520Inasey3TAyqLyojQa8MnmCALSEU29yQFW8dePZ7xQTvSt73pHazLFTK5f7SyLUJSo2uKLesEtEa9aUYcgMAAACPeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IkRpc2FibGVUaGlyZFBhcnR5U3RvcmFnZVBhcnRpdGlvbmluZyIsImV4cGlyeSI6MTcyNTQwNzk5OSwiaXNTdWJkb21haW4iOnRydWUsImlzVGhpcmRQYXJ0eSI6dHJ1ZX0=" http-equiv="origin-trial"/>
<link href="https://cdn.cookielaw.org" rel="preconnect"/>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-V649BZ47DJ&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" crossorigin="anonymous" integrity="sha384-wEVSdqKc5hf9vkWC9kAmVRAEa11o8QNGecO6p5G2xGj+RmQhgDqFDCRdpfSr1VUf" src="https://www.gstatic.com/recaptcha/releases/moV1mTgQ6S91nuTnmll4Y9yf/recaptcha__en.js" type="text/javascript"></script><script async="" src="//resources.planetnine.com/player/new/Pnvp.js?t=0.684509417726062&amp;h=www.pararius.com"></script><script async="" src="https://w

In [ ]:
def extract(driver, page_source) -> dict:
    # Use BeautifulSoup to parse the page source and extract the data you need
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find all the links to the individual listings
    listing_links = soup.select('.listing-search-item__link')

    houses = []

    # Iterate over each link
    for link in listing_links:
        # Get the URL of the individual listing
        listing_url = link['href']

        # Navigate to the individual listing page
        driver.get(listing_url)

        # Wait for the page to load
        time.sleep(5)

        # Get the page source of the individual listing page
        listing_page_source = driver.page_source

        # Parse the individual listing page
        listing_soup = BeautifulSoup(listing_page_source, 'html.parser')

        # Extract the information you need from the individual listing page
        # For example, let's say you want to extract the description of the house
        description = listing_soup.select_one('.listing-detail-summary').text

        # Add the extracted information to the houses list
        houses.append({
            'url': listing_url,
            'description': description,
        })

    print(f"This is kado: \n{houses}")

def parse():
    # ...
    # Extract recipes from the page source
    houses = extract(driver, page_source)
    # ...


In [8]:
import datetime
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

def parse():
    base_url = "https://www.pararius.com/apartments/amsterdam"

    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode to avoid opening a browser window
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())

    try:
        print("Getting driver...")
        # Launch the Chrome browser with the configured options
        driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)
        print("Done!")
    except Exception as err:
        print("Encountered an exception!")
        raise err

    time.sleep(5)
    # Navigate to the URL
    driver.get(base_url)

    # Use explicit wait to wait for recipe elements to load
    wait = WebDriverWait(driver, 7)
    time.sleep(10)

    # Get the page source (which will contain the dynamically generated content)
    page_source = driver.page_source

    # Extract houses from the page source
    houses = extract(driver, page_source)

    driver.quit() # close driver

def extract(driver, page_source) -> dict:
    # Use BeautifulSoup to parse the page source and extract the data you need
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find all the links to the individual listings
    listing_links = soup.select('.listing-search-item__link')

    houses = []

    # listings = []
    # for listing_section in soup.find_all('section', class_='listing-search-item listing-search-item--list listing-search-item--for-rent'):

    #     # Title and Link
    #     title_link = listing_section.find('a', class_='listing-search-item__link listing-search-item__link--title')
    #     if title_link:
    #         title = title_link.text.strip()
    #         link = title_link['href']

    #     # Address
    #     sub_title = listing_section.find('div', class_='listing-search-item__sub-title')
    #     if sub_title:
    #         address = sub_title.text.strip()

    #     # Price
    #     price = listing_section.find('div', class_='listing-search-item__price')
    #     if price:
    #         price = price.text.strip()

    #     # Features (area, rooms, etc.)
    #     features = []
    #     feature_items = listing_section.find_all('li', class_='illustrated-features__item')
    #     for item in feature_items:
    #         features.append(item.text.strip())

    #     # Agent (if available)
    #     agent_link = listing_section.find('a', class_='listing-search-item__link')
    #     if agent_link:
    #         agent_name = agent_link.text.strip()

    #     # Add all the extracted information to a dictionary
    #     listing_data = {
    #         'title': title,
    #         'link': link,
    #         'address': address,
    #         'price': price,
    #         'features': features,
    #         'agent_name': agent_name if agent_name else None
    #     }
    #     listings.append(listing_data)

# Now you have a list of dictionaries containing information for each listing
# print(listings)


    # Iterate over each link
    for link in listing_links:
        # Get the URL of the individual listing
        listing_url = link['href']

        # Navigate to the individual listing page
        driver.get(listing_url)

        # Wait for the page to load
        time.sleep(5)

        # Get the page source of the individual listing page
        listing_page_source = driver.page_source

        # Parse the individual listing page
        listing_soup = BeautifulSoup(listing_page_source, 'html.parser')

        # Extract the information you need from the individual listing page
        description = listing_soup.select_one('.listing-detail-summary').text

        # Add the extracted information to the houses list
        houses.append({
            'url': listing_url,
            'description': description,
        })

    print(f"This is kado: \n{houses}")

if __name__ == "__main__":
    parse()

Getting driver...
Done!


InvalidArgumentException: Message: invalid argument
  (Session info: chrome-headless-shell=121.0.6167.184)
Stacktrace:
#0 0x5c8499fbf303 <unknown>
#1 0x5c8499ca40f7 <unknown>
#2 0x5c8499c8bc6f <unknown>
#3 0x5c8499c89bb2 <unknown>
#4 0x5c8499c8a11a <unknown>
#5 0x5c8499ca6aee <unknown>
#6 0x5c8499d31cc5 <unknown>
#7 0x5c8499d12382 <unknown>
#8 0x5c8499d3107c <unknown>
#9 0x5c8499d12123 <unknown>
#10 0x5c8499ce2095 <unknown>
#11 0x5c8499ce309e <unknown>
#12 0x5c8499f836ab <unknown>
#13 0x5c8499f874ba <unknown>
#14 0x5c8499f6ff85 <unknown>
#15 0x5c8499f8812f <unknown>
#16 0x5c8499f53e6f <unknown>
#17 0x5c8499fac5f8 <unknown>
#18 0x5c8499fac7c2 <unknown>
#19 0x5c8499fbe4a4 <unknown>
#20 0x733034a94ac3 <unknown>


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

base_url = "https://www.pararius.com/apartments/amsterdam"
# Replace with the URL you get from BeautifulSoup results (e.g., /apartment-for-rent/amsterdam/ac6a3132/van-nijenrodeweg)
listing_url = "/apartment-for-rent/amsterdam/ac6a3132/van-nijenrodeweg"

# driver = webdriver.Chrome(service=ChromeDriverManager().install())
# Install the appropriate chromedriver automatically based on your Selenium version
webdriver_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=webdriver_service)
driver.get(base_url)

# Wait for content to load (replace with a specific waiting condition if needed)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "property-details")))

# Option 1: Get the complete URL from the browser address bar
final_url = driver.current_url

# Option 2: Extract the final URL from a specific element (replace with appropriate selectors)
# soup = BeautifulSoup(driver.page_source, 'html.parser')
# final_url = soup.find("a", class_="listing-search-item__link listing-search-item__link--title")["href"]

# Use the final_url for further processing or data extraction

# ... your code to extract details from the listing page

driver.quit()


TimeoutException: Message: 
Stacktrace:
#0 0x5b2fa5fec303 <unknown>
#1 0x5b2fa5cd12b7 <unknown>
#2 0x5b2fa5d1ccc0 <unknown>
#3 0x5b2fa5d1cdb1 <unknown>
#4 0x5b2fa5d60b44 <unknown>
#5 0x5b2fa5d3f3ad <unknown>
#6 0x5b2fa5d5e07c <unknown>
#7 0x5b2fa5d3f123 <unknown>
#8 0x5b2fa5d0f095 <unknown>
#9 0x5b2fa5d1009e <unknown>
#10 0x5b2fa5fb06ab <unknown>
#11 0x5b2fa5fb44ba <unknown>
#12 0x5b2fa5f9cf85 <unknown>
#13 0x5b2fa5fb512f <unknown>
#14 0x5b2fa5f80e6f <unknown>
#15 0x5b2fa5fd95f8 <unknown>
#16 0x5b2fa5fd97c2 <unknown>
#17 0x5b2fa5feb4a4 <unknown>
#18 0x7f677e294ac3 <unknown>


In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def extract_listing_url(base_url: str) -> str:
  """
  Extracts the final URL with the JavaScript token extension from a Pararius listing page.

  Args:
      base_url (str): The base URL of the Pararius listing page.

  Returns:
      str: The complete URL with the JavaScript token extension, or None if the element is not found.
  """

  try:
    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=webdriver_service)

    driver.get(base_url)

    # Wait for the element containing the link to load (replace with a specific condition if needed)
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "h2.listing-search-item__title a.listing-search-item__link.listing-search-item__link--title")
        )
    )

    # Extract the href attribute containing the URL with the JavaScript token
    href_with_token = element.get_attribute("href")

    return href_with_token

  except Exception as e:
    print(f"Error extracting URL: {e}")
    return None

  finally:
    driver.quit()

# Get the final URL with the JavaScript token extension
base_url = "https://www.pararius.com/apartments/amsterdam"
final_url = extract_listing_url(base_url)

if final_url:
  print("Extracted URL:", final_url)
else:
  print("Failed to extract URL for", base_url)

# You can use the final_url for further processing or data extraction
# ... your code to extract details from the listing page ...


Extracted URL: https://www.pararius.com/apartment-for-rent/amsterdam/18c66a74/keizersgracht


In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

base_url = "https://www.pararius.com/apartments/amsterdam"


def extract_listing_urls(search_url):
  """
  Extracts URLs with JavaScript tokens for all listings on a Pararius search page.

  Args:
      search_url (str): The URL of the Pararius search page.

  Returns:
      list: A list of URLs with JavaScript tokens for all listings on the page.
  """

  listing_urls = []

  try:
    webdriver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=webdriver_service)

    driver.get(search_url)

    # Wait for the initial content to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "search-results")))

    # Find all elements containing listing titles (may need adjustment based on page structure)
    listing_titles = driver.find_elements(By.CSS_SELECTOR, "h2.listing-search-item__title")

    for title in listing_titles:
      # Extract the link element from the title element (replace with appropriate selector if needed)
      link = title.find_element(By.TAG_NAME, "a")
      listing_url = link.get_attribute("href")
      listing_urls.append(listing_url)

    return listing_urls

  except Exception as e:
    print(f"Error extracting URLs: {e}")
    return []

  finally:
    driver.quit()


def extract_listing_features(listing_url):
  """
  Extracts specific features from a Pararius listing page (example implementation).

  Args:
      listing_url (str): The URL of the Pararius listing page with JavaScript token.

  Returns:
      dict: A dictionary containing extracted features (example structure).
  """

  features = {}

  try:
    webdriver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=webdriver_service)

    driver.get(listing_url)

    # Wait for the listing content to load (replace with specific conditions for each feature)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "property-details")))

    # Example feature extraction using BeautifulSoup (replace with desired selectors)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract title (example)
    title_element = soup.find("h1", class_="listing-details__title")
    features["title"] = title_element.text.strip() if title_element else None

    # Extract additional features (replace with your specific logic)
    # ... feature extraction code for area, price, etc. ...

    return features

  except Exception as e:
    print(f"Error extracting features for {listing_url}: {e}")
    return {}

  finally:
    driver.quit()


# Example usage:
search_url = f"{base_url}/rent"  # Modify for specific search criteria
all_listing_urls = extract_listing_urls(search_url)

if all_listing_urls:
  print("Extracted listing URLs:")
  for url in all_listing_urls:
    print(url)

  # Iterate through each listing URL and extract features
  for url in all_listing_urls:
    listing_features = extract_listing_features(url)
    if listing_features:
      print(f"Features for {url}:")
      for key, value in listing_features.items():
        print(f"\t- {key}: {value}")
else:
  print("Failed to extract listing URLs")


Error extracting URLs: Message: 
Stacktrace:
#0 0x592649c15303 <unknown>
#1 0x5926498fa2b7 <unknown>
#2 0x592649945cc0 <unknown>
#3 0x592649945db1 <unknown>
#4 0x592649989b44 <unknown>
#5 0x5926499683ad <unknown>
#6 0x59264998707c <unknown>
#7 0x592649968123 <unknown>
#8 0x592649938095 <unknown>
#9 0x59264993909e <unknown>
#10 0x592649bd96ab <unknown>
#11 0x592649bdd4ba <unknown>
#12 0x592649bc5f85 <unknown>
#13 0x592649bde12f <unknown>
#14 0x592649ba9e6f <unknown>
#15 0x592649c025f8 <unknown>
#16 0x592649c027c2 <unknown>
#17 0x592649c144a4 <unknown>
#18 0x7b2603294ac3 <unknown>

Failed to extract listing URLs


In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def extract_listing_url(base_url: str) -> str:
  """
  Extracts the final URL with the JavaScript token extension from a Pararius listing page.

  Args:
      base_url (str): The base URL of the Pararius listing page.

  Returns:
      str: The complete URL with the JavaScript token extension, or None if the element is not found.
  """

  try:
    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=webdriver_service)

    driver.get(base_url)

    # Wait for the element containing the link to load (replace with a specific condition if needed)
    elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "h2.listing-search-item__title a.listing-search-item__link.listing-search-item__link--title")
        )
    )

    # Extract URLs from all elements and store them in a list
    listing_urls = []
    for element in elements:
      href_with_token = element.get_attribute("href")
      listing_urls.append(href_with_token)

    return listing_urls

  except Exception as e:
    print(f"Error extracting URLs: {e}")
    return None

  finally:
    driver.quit()

# Get all listing URLs from the Pararius search page
base_url = "https://www.pararius.com/apartments/amsterdam"
listing_urls = extract_listing_url(base_url)

if listing_urls:
  print(f"Extracted {len(listing_urls)} listing URLs:")
  for url in listing_urls:
    print(url)

  # Example: Access each listing URL for further data extraction
#   for url in listing_urls:
    # Use the extracted URL to open the individual listing page and extract details
    # ... your code to extract details from each listing page ...
else:
  print("Failed to extract listing URLs.")


Extracted 32 listing URLs:
https://www.pararius.com/apartment-for-rent/amsterdam/15dbf14f/westerdok
https://www.pararius.com/apartment-for-rent/amsterdam/a51ed7b8/cruquiusweg
https://www.pararius.com/apartment-for-rent/amsterdam/049f30fe/amazonenstraat
https://www.pararius.com/apartment-for-rent/amsterdam/624faf4a/vondelstraat
https://www.pararius.com/apartment-for-rent/amsterdam/2139c2d1/nieuwendijk
https://www.pararius.com/apartment-for-rent/amsterdam/0df2ff24/molukkenstraat
https://www.pararius.com/apartment-for-rent/amsterdam/6e4ed538/gustav-mahlerlaan
https://www.pararius.com/apartment-for-rent/amsterdam/53a7db70/jacob-van-lennepstraat
https://www.pararius.com/apartment-for-rent/amsterdam/dcdd4e66/assumburg
https://www.pararius.com/apartment-for-rent/amsterdam/8e5e1751/weteringschans
https://www.pararius.com/apartment-for-rent/amsterdam/f61bb14f/keizersgracht
https://www.pararius.com/apartment-for-rent/amsterdam/509a18ae/palestrinastraat
https://www.pararius.com/apartment-for-rent

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def extract_listing_urls(base_url: str) -> list[str]:
  """
  Extracts all listing URLs from the given Pararius search page and subsequent pages.

  Args:
      base_url (str): The base URL of the Pararius search page (e.g., https://www.pararius.com/apartments/amsterdam).

  Returns:
      list[str]: A list of all extracted listing URLs, or an empty list if there's an error.
  """

  all_listing_urls = []
  current_url = base_url

  try:
    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=webdriver_service)

    while True:
      driver.get(current_url)

      # Wait for the element containing links to load (replace with a specific condition if needed)
      elements = WebDriverWait(driver, 10).until(
          EC.presence_of_all_elements_located(
              (By.CSS_SELECTOR, "h2.listing-search-item__title a.listing-search-item__link.listing-search-item__link--title")
          )
      )

      # Extract URLs from all elements on the current page
      listing_urls = []
      for element in elements:
        href_with_token = element.get_attribute("href")
        listing_urls.append(href_with_token)

      all_listing_urls.extend(listing_urls)  # Add current page URLs to the master list

      # Check for the "Next" pagination link and break if not found
      try:
        next_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "li.pagination__item.pagination__item--next a.pagination__link.pagination__link--next")
            )
        )
      except EC.TimeoutException:
        break

      # Construct the URL for the next page based on the data-page attribute
      current_page = int(next_button.get_attribute("data-page"))
      next_url = f"{base_url}/page-{current_page + 1}"

      current_url = next_url  # Update current_url for the next iteration

  except Exception as e:
    print(f"Error extracting listing URLs: {e}")
    return []

  finally:
    driver.quit()

  return all_listing_urls

# Get all listing URLs from all pages of the Pararius search
base_url = "https://www.pararius.com/apartments/amsterdam"
listing_urls = extract_listing_urls(base_url)

if listing_urls:
  print(f"Extracted {len(listing_urls)} listing URLs in total:")
  for url in listing_urls:
    print(url)
else:
  print("Failed to extract listing URLs.")


Error extracting listing URLs: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.184)
Stacktrace:
#0 0x5fda86ff0303 <unknown>
#1 0x5fda86cd52b7 <unknown>
#2 0x5fda86cab958 <unknown>
#3 0x5fda86d4bded <unknown>
#4 0x5fda86d619d9 <unknown>
#5 0x5fda86d43123 <unknown>
#6 0x5fda86d13095 <unknown>
#7 0x5fda86d1409e <unknown>
#8 0x5fda86fb46ab <unknown>
#9 0x5fda86fb84ba <unknown>
#10 0x5fda86fa0f85 <unknown>
#11 0x5fda86fb912f <unknown>
#12 0x5fda86f84e6f <unknown>
#13 0x5fda86fdd5f8 <unknown>
#14 0x5fda86fdd7c2 <unknown>
#15 0x5fda86fef4a4 <unknown>
#16 0x7d284e694ac3 <unknown>

Failed to extract listing URLs.


In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def extract_listing_urls(base_url: str, max_pages: int = None) -> list[str]:
  """
  Extracts all listing URLs from the given Pararius search page and subsequent pages,
  with an optional limit on the number of pages to iterate through.

  Args:
      base_url (str): The base URL of the Pararius search page (e.g., https://www.pararius.com/apartments/amsterdam).
      max_pages (int, optional): The maximum number of pages to extract URLs from. Defaults to None (extract all pages).

  Returns:
      list[str]: A list of all extracted listing URLs, or an empty list if there's an error.
  """

  all_listing_urls = []
  current_url = base_url
  page_number = 1

  try:
    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=webdriver_service)

    while True:
      driver.get(current_url)

      # Wait for the element containing links to load (replace with a specific condition if needed)
      elements = WebDriverWait(driver, 10).until(
          EC.presence_of_all_elements_located(
              (By.CSS_SELECTOR, "h2.listing-search-item__title a.listing-search-item__link.listing-search-item__link--title")
          )
      )

      # Extract URLs from all elements on the current page
      listing_urls = []
      for element in elements:
        href_with_token = element.get_attribute("href")
        listing_urls.append(href_with_token)

      all_listing_urls.extend(listing_urls)  # Add current page URLs to the master list

      # Check for the "Next" pagination link and break if not found or max pages reached
      try:
        next_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "li.pagination__item.pagination__item--next a.pagination__link.pagination__link--next")
            )
        )
      except EC.TimeoutException:
        break

      if max_pages is not None and page_number >= max_pages:
        break

      # Construct the URL for the next page based on the data-page attribute
      current_page = int(next_button.get_attribute("data-page"))
      next_url = f"{base_url}/page-{current_page + 1}"

      current_url = next_url
      page_number += 1  # Increment page counter

  except Exception as e:
    print(f"Error extracting listing URLs: {e}")
    return []

  finally:
    driver.quit()

  return all_listing_urls

# Get all listing URLs from all pages of the Pararius search (or up to 5 pages)
base_url = "https://www.pararius.com/apartments/amsterdam"
listing_urls = extract_listing_urls(base_url, max_pages=10)  # Limit to 5 pages

if listing_urls:
  print(f"Extracted {len(listing_urls)} listing URLs in total:")
  for url in listing_urls:
    print(url)
else:
  print("Failed to extract listing URLs.")


Extracted 320 listing URLs in total:
https://www.pararius.com/apartment-for-rent/amsterdam/15dbf14f/westerdok
https://www.pararius.com/apartment-for-rent/amsterdam/9818a7d4/haparandaweg
https://www.pararius.com/apartment-for-rent/amsterdam/b65f5534/august-allebeplein
https://www.pararius.com/apartment-for-rent/amsterdam/d7c50e79/august-allebeplein
https://www.pararius.com/apartment-for-rent/amsterdam/bd2a9300/jan-evertsenstraat
https://www.pararius.com/apartment-for-rent/amsterdam/049f30fe/amazonenstraat
https://www.pararius.com/apartment-for-rent/amsterdam/624faf4a/vondelstraat
https://www.pararius.com/apartment-for-rent/amsterdam/2139c2d1/nieuwendijk
https://www.pararius.com/apartment-for-rent/amsterdam/0df2ff24/molukkenstraat
https://www.pararius.com/apartment-for-rent/amsterdam/6e4ed538/gustav-mahlerlaan
https://www.pararius.com/apartment-for-rent/amsterdam/53a7db70/jacob-van-lennepstraat
https://www.pararius.com/apartment-for-rent/amsterdam/dcdd4e66/assumburg
https://www.pararius.

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def extract_listing_urls(base_url: str, max_pages: int = None) -> list[str]:
  """
  Extracts all listing URLs from the given Pararius search page and subsequent pages,
  with an optional limit on the number of pages to iterate through.

  Args:
      base_url (str): The base URL of the Pararius search page (e.g., https://www.pararius.com/apartments/amsterdam).
      max_pages (int, optional): The maximum number of pages to extract URLs from. Defaults to None (extract all pages).

  Returns:
      list[str]: A list of all extracted listing URLs, or an empty list if there's an error.
  """

  all_listing_urls = []
  current_url = base_url
  page_number = 1

  try:
    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=webdriver_service)

    while True:
      driver.get(current_url)

      # Wait for the element containing links to load (replace with a specific condition if needed)
      elements = WebDriverWait(driver, 10).until(
          EC.presence_of_all_elements_located(
              (By.CSS_SELECTOR, "h2.listing-search-item__title a.listing-search-item__link.listing-search-item__link--title")
          )
      )

      # Extract URLs from all elements on the current page
      listing_urls = []
      for element in elements:
        href_with_token = element.get_attribute("href")
        listing_urls.append(href_with_token)

      all_listing_urls.extend(listing_urls)  # Add current page URLs to the master list

      # Check for the "Next" pagination link and break if not found or max pages reached
      try:
        next_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "li.pagination__item.pagination__item--next a.pagination__link.pagination__link--next")
            )
        )
      except EC.TimeoutException:
        break

      if max_pages is not None and page_number >= max_pages:
        break

      # Construct the URL for the next page based on the data-page attribute
      current_page = int(next_button.get_attribute("data-page"))
      next_url = f"{base_url}/page-{current_page + 1}"

      current_url = next_url
      page_number += 1  # Increment page counter

  except Exception as e:
    print(f"Error extracting listing URLs: {e}")
    return []

  finally:
    driver.quit()

  return all_listing_urls

def extract_listing_features(listing_url: str) -> dict[str, str]:
  """
  Extracts desired features from a Pararius listing page using BeautifulSoup.

  Args:
      listing_url (str): The URL of the Pararius listing page.

  Returns:
      dict[str, str]: A dictionary containing extracted features (key: feature name, value: feature value).
  """

  features = {}

  try:
    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=webdriver_service)

    driver.get(listing_url)

    # Wait for the element containing features (replace with a specific condition if needed)
    feature_section = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "section.page__details.page__details--transfer"))
    )

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(feature_section.get_attribute("innerHTML"), "html.parser")

    # Extract features
    for feature_item in soup.find_all("dt", class_="listing-features__term"):
      feature_name = feature_item.text.strip()
      feature_value = feature_item.find_next_sibling("dd").find("span", class_="listing-features__main-description").text.strip()
      features[feature_name] = feature_value

  except Exception as e:
    print(f"Error extracting features for URL {listing_url}: {e}")
    return {}

  finally:
    driver.quit()

  return features

# Get all listing URLs from all pages of the Pararius search (or up to 5 pages)
base_url = "https://www.pararius.com/apartments/amsterdam"
listing_urls = extract_listing_urls(base_url, max_pages=1)

if listing_urls:
  print(f"Extracted {len(listing_urls)} listing URLs:")

  # Extract features from each listing URL
  for i, url in enumerate(listing_urls):
    print(f"Processing URL {i + 1}/{len(listing_urls)}: {url}")
    features = extract_listing_features(url)

    if features:
      print(f"Features for URL {url}:")
      for feature_name, feature_value in features.items():
        print(f"- {feature_name}: {feature_value}")
    else:
      print(f"Failed to extract features for URL {url}.")

    # Break the loop if this is the last URL
    if i == len(listing_urls) - 1:
      break

else:
  print("Failed to extract listing URLs.")


Extracted 32 listing URLs:
Processing URL 1/32: https://www.pararius.com/apartment-for-rent/amsterdam/15dbf14f/westerdok
Features for URL https://www.pararius.com/apartment-for-rent/amsterdam/15dbf14f/westerdok:
- Rental price: €3,495 per month
- Offered since: 5 weeks
- Status: For rent
- Available: Immediately
- Deposit: €6,990
- Interior: Furnished
Processing URL 2/32: https://www.pararius.com/apartment-for-rent/amsterdam/68f3556f/martelaarsgracht
Features for URL https://www.pararius.com/apartment-for-rent/amsterdam/68f3556f/martelaarsgracht:
- Rental price: €2,350 per month
- Offered since: 6 weeks
- Status: For rent
- Available: Immediately
- Rental agreement: Temporary rental
- Deposit: €4,700
- Interior: Furnished
- Upkeep: Good
Processing URL 3/32: https://www.pararius.com/apartment-for-rent/amsterdam/4a6e3bdd/leerdamhof


KeyboardInterrupt: 

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def extract_listing_urls(base_url: str, max_pages: int = None) -> list[str]:
  """
  Extracts all listing URLs from the given Pararius search page and subsequent pages,
  with an optional limit on the number of pages to iterate through.

  Args:
      base_url (str): The base URL of the Pararius search page (e.g., https://www.pararius.com/apartments/amsterdam).
      max_pages (int, optional): The maximum number of pages to extract URLs from. Defaults to None (extract all pages).

  Returns:
      list[str]: A list of all extracted listing URLs, or an empty list if there's an error.
  """

  all_listing_urls = []
  current_url = base_url
  page_number = 1

  try:
    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=webdriver_service)

    while True:
      driver.get(current_url)

      # Wait for the element containing links to load (replace with a specific condition if needed)
      elements = WebDriverWait(driver, 10).until(
          EC.presence_of_all_elements_located(
              (By.CSS_SELECTOR, "h2.listing-search-item__title a.listing-search-item__link.listing-search-item__link--title")
          )
      )

      # Extract URLs from all elements on the current page
      listing_urls = []
      for element in elements:
        href_with_token = element.get_attribute("href")
        listing_urls.append(href_with_token)

      all_listing_urls.extend(listing_urls)  # Add current page URLs to the master list

      # Check for the "Next" pagination link and break if not found or max pages reached
      try:
        next_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "li.pagination__item.pagination__item--next a.pagination__link.pagination__link--next")
            )
        )
      except EC.TimeoutException:
        break

      if max_pages is not None and page_number >= max_pages:
        break

      # Construct the URL for the next page based on the data-page attribute
      current_page = int(next_button.get_attribute("data-page"))
      next_url = f"{base_url}/page-{current_page + 1}"

      current_url = next_url
      page_number += 1  # Increment page counter

  except Exception as e:
    print(f"Error extracting listing URLs: {e}")
    return []

  finally:
    driver.quit()

  return all_listing_urls

def extract_listing_features(listing_url: str) -> dict[str, str]:
  """
  Extracts desired features from a Pararius listing page using BeautifulSoup.

  Args:
      listing_url (str): The URL of the Pararius listing page.

  Returns:
      dict[str, str]: A dictionary containing extracted features (key: feature name, value: feature value).
  """

  features = {}

  try:
    # Install the appropriate chromedriver automatically based on your Selenium version
    webdriver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=webdriver_service)

    driver.get(listing_url)

    # Wait for the element containing features (replace with a specific condition if needed)
    feature_section = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "section.page__details.page__details--dimensions"))
    )

    # Page source 
    page_source = driver.page_source

    # Use BeautifulSoup to parse the HTML content
    # soup = BeautifulSoup(feature_section.get_attribute("innerHTML"), "html.parser")

    soup = BeautifulSoup(page_source, 'html.parser')

    extract_features_from_section(soup, features)

  except Exception as e:
    print(f"Error extracting features for URL {listing_url}: {e}")
    return {}

  finally:
    driver.quit()

  return features

def extract_features_from_section(soup: BeautifulSoup, features: dict[str, str] = {}) -> dict[str, str]:
    """
    Extracts features from a specific section of a Pararius listing page using BeautifulSoup.

    Args:
        soup (BeautifulSoup): The BeautifulSoup object containing the HTML content.
        features (dict[str, str], optional): A dictionary to store the extracted features. Defaults to {}.

    Returns:
        dict[str, str]: A dictionary containing the extracted features.
    """

    kado = soup.find_all("article", class_="page__row page__row--listing")

    print(type(kado[0]))
    # sections = kado[0].find_all("section")

    # Extract features
    # for listing_features in kado:
    #   print(listing_features)
      # feature_item = listing_features.find_all("dt", class_="listing-features__term")
      # feature_name = feature_item.text.strip()
      # feature_value = feature_item.find_next_sibling("dd").find("span", class_="listing-features__main-description").text.strip()
      # features[feature_name] = feature_value
1
# Get all listing URLs from all pages of the Pararius search (or up to 5 pages)
base_url = "https://www.pararius.com/apartments/amsterdam"
listing_urls = extract_listing_urls(base_url, max_pages=1)

if listing_urls:
  print(f"Extracted {len(listing_urls)} listing URLs:")

  # Extract features from each listing URL
  for i, url in enumerate(listing_urls):
    print(f"Processing URL {i + 1}/{len(listing_urls)}: {url}")
    features = extract_listing_features(url)

    if features:
      print(f"Features for URL {url}:")
      for feature_name, feature_value in features.items():
        print(f"- {feature_name}: {feature_value}")
    else:
      print(f"Failed to extract features for URL {url}.")

    # Break the loop if this is the last URL
    if i == len(listing_urls) - 1:
      break

else:
  print("Failed to extract listing URLs.")


Extracted 32 listing URLs:
Processing URL 1/32: https://www.pararius.com/apartment-for-rent/amsterdam/a51ed7b8/cruquiusweg
<class 'bs4.element.Tag'>
Failed to extract features for URL https://www.pararius.com/apartment-for-rent/amsterdam/a51ed7b8/cruquiusweg.
Processing URL 2/32: https://www.pararius.com/apartment-for-rent/amsterdam/15dbf14f/westerdok


KeyboardInterrupt: 